## Домашнее задание к занятию 2: "Масштабирование признаков. L1- и L2-регуляризация. Стохастический градиентный спуск"

1. Сгенерировать датасет при помощи sklearn.datasets.make_regression и обучить линейную модель при помощи градиентного и стохастического градиентного спуска. Нанести среднеквадратичную ошибку для обоих методов на один график, сделать выводы о разнице скорости сходимости каждого из методов.

In [ ]:
from sklearn.datasets import make_regression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X, y, coef = make_regression(n_samples=100, n_features=10, coef = True)

In [ ]:
X_for_grad = X.T

In [ ]:
X.shape

In [ ]:
y

In [ ]:
def mserror(X, w, y_pred):
    y = X.dot(w)
    return (sum((y - y_pred)**2)) / len(y)

In [ ]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [ ]:
#Градиентный спуск
errors_grad = []
W = np.zeros(10)
w_list = [W.copy()]
alpha = 0.2
n = X.shape[1]
for i in range(100):
    y_pred = np.dot(W, X_for_grad)
    err = calc_mse(y, y_pred)
    W -= alpha * (2/n * np.sum((X_for_grad * (y_pred - y)), axis=1)) 
    w_list.append(W)
    errors_grad.append(err)
        
w_list_grad = np.array(w_list)
errors_grad = np.array(errors_grad)

In [ ]:
# Стохастический градиентный спуск
w = np.zeros(10)
w_list = [w.copy()]
errors = []
eta = 0.5
max_iter = 1e5
min_weight_dist = 1e-8
weight_dist = np.inf
iter_num = 0
 
# ход градиентного спуска
while weight_dist > min_weight_dist and iter_num < max_iter:    
    new_w = w - 2 * eta * np.dot(X.T, (np.dot(X, w) - y)) / y.shape[0]
    weight_dist = np.linalg.norm(new_w - w, ord=2)
    w_list.append(new_w.copy())
    errors.append(mserror(X, new_w, y))
    iter_num += 1
    w = new_w
w_list_stochastic = np.array(w_list)

In [ ]:
# Визуализируем изменение функционала ошибки
plt.plot(range(len(errors)), errors, label='Стохастический градиентный спуск', linewidth = 4)
plt.plot(range(len(errors_grad)), errors_grad, label='Градиентный спуск', linewidth = 2)
plt.xlim(-1, 100)
plt.title('MSE')
plt.xlabel('Iteration number')
plt.ylabel('MSE')
plt.legend()

Вывод: Стохастический градиентный спуск сходится быстрее обычного градиентного спуска. 